This note book contains the scripts for preprocessing financil data from a video, converting them from video to frames and then sending them to VERTEX GEMINI API for extracting data. using offcial Vertex SDK for video uploading.

In [ ]:
# # only run if you want to To download video from s3 or any downloadable link convert video into frames and save it into a folder. read the last 2 instruction paras
# import cv2
# import os
# import urllib.request

# def download_video(url, local_path):
#     try:
#         urllib.request.urlretrieve(url, local_path)
#         print(f"Video downloaded from {url} to: {local_path}")
#         return True
#     except Exception as e:
#         print(f"Error downloading video: {e}")
#         return False

# def save_frames_from_video(video_path, output_folder, frame_interval=1):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     video = cv2.VideoCapture(video_path)
#     if not video.isOpened():
#         print("Error: Could not open video.")
#         return

#     frame_count = 0
#     while True:
#         success, frame = video.read()
#         if not success:
#             break

#         if frame_count % frame_interval == 0:
#             frame_file = os.path.join(output_folder, f"frame_{frame_count}.jpg")
#             cv2.imwrite(frame_file, frame)

#         frame_count += 1

#     video.release()
#     print(f"Total frames saved: {frame_count}")

# # System prompt for video URL
# video_url = input("Enter the URL of the video to download: ")
# local_video_path = "downloaded_video.mp4"  # Local path to save the downloaded video

# # User prompt for frame extraction interval
# frame_interval = int(input("Enter the frame extraction interval (e.g., 1 for every frame, 10 for every 10th frame, etc.): "))

# # Download video from the provided URL
# if download_video(video_url, local_video_path):
#     # Process and save frames from the downloaded video
#     output_directory = "./2Day frames"  # Replace with your desired output folder
#     save_frames_from_video(local_video_path, output_directory)
    
# # # Video URL
# # video_url = "https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/SPX_2D.mp4"
# # local_video_path = "2day Data.mp4"  # Local path to save the downloaded video

# # Download video from the provided URL
# if download_video(video_url, local_video_path):
#     # Process and save frames from the downloaded video
#     output_directory = "./2Day frames"  # Replace with your desired output folder
#     save_frames_from_video(local_video_path, output_directory)


In [ ]:
import cv2
import os
import urllib.request

def download_video(url, local_path):
    if os.path.exists(local_path):
        print(f"Video already exists at: {local_path}")
        return True

    try:
        urllib.request.urlretrieve(url, local_path)
        print(f"Video downloaded from {url} to: {local_path}")
        return True
    except Exception as e:
        print(f"Error downloading video: {e}")
        return False

def save_frames_from_video(video_path, output_folder, frame_interval=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    while True:
        success, frame = video.read()
        if not success:
            break

        if frame_count % frame_interval == 0:
            frame_file = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_file, frame)

        frame_count += 1

    video.release()
    print(f"Total frames saved: {frame_count}")

# System prompt for video URL
video_url = input("Enter the URL of the video to download: ")
local_video_path = "downloaded_video.mp4"  # Local path to save the downloaded video

# User prompt for frame extraction interval
frame_interval = int(input("Enter the frame extraction interval (e.g., 1 for every frame, 10 for every 10th frame, etc.): "))

# Download video from the provided URL if not already downloaded
if download_video(video_url, local_video_path):
    # Process and save frames from the downloaded video
    output_directory = "./2Day frames"  # Replace with your desired output folder
    save_frames_from_video(local_video_path, output_directory, frame_interval)


In [ ]:
!pip install pytesseract

In [ ]:
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = (
#     r'/usr/bin/tesseract'
# )

In [ ]:
import os
from PIL import Image
import pytesseract


# Path to the folder containing images
folder_path = '2Day frames'

# This function uses Tesseract to extract text from a specified region in an image
def extract_date_from_image(image_path, region):
    with Image.open(image_path) as img:
        cropped_img = img.crop(region)
        text = pytesseract.image_to_string(cropped_img, lang='eng', config='--psm 6')
        return text.strip()


# Define the region where the date is located
date_region = (142, 50, 267, 75)

# Get all image file paths
image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('.jpg', '.png'))]

# Create a dictionary to group images by date
date_groups = {}

# Process each image
for image_path in image_paths:
    date_text = extract_date_from_image(image_path, date_region)
    if date_text not in date_groups:
        date_groups[date_text] = []
    date_groups[date_text].append(image_path)

# Now you have a dictionary where the key is the date and the value is a list of image paths with that date
for date, images in date_groups.items():
    print(f"Date: {date}, Images: {images}")




In [ ]:
#ABOVE CODE WITH 10 MIN TIMEOUT, USE THIS BLOCK IF PROCESSING IS SLOW ABOVE
import os
from PIL import Image
import pytesseract
import time

# Path to the folder containing images
folder_path = '2Day frames'

# This function uses Tesseract to extract text from a specified region in an image
def extract_date_from_image(image_path, region):
    try:
        with Image.open(image_path) as img:
            cropped_img = img.crop(region)
            text = pytesseract.image_to_string(cropped_img, lang='eng', config='--psm 6')
            return text.strip()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Define the region where the date is located
date_region = (142, 50, 267, 75)

# Get all image file paths
image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('.jpg', '.png'))]

# Create a dictionary to group images by date
date_groups = {}

# Set a timeout (in seconds)
timeout = 600  # 10 minutes

# Process each image
start_time = time.time()
for image_path in image_paths:
    elapsed_time = time.time() - start_time
    if elapsed_time > timeout:
        print(f"Timeout reached. Exiting.")
        break

    date_text = extract_date_from_image(image_path, date_region)
    if date_text is not None:
        if date_text not in date_groups:
            date_groups[date_text] = []
        date_groups[date_text].append(image_path)

# Now you have a dictionary where the key is the date and the value is a list of image paths with that date
for date, images in date_groups.items():
    print(f"Date: {date}, Images: {images}")


In [ ]:
new = {}
for date, images in date_groups.items():
    if images:  # Check if there are any images for the date
        new[date] = images[0]  # Add only the first image of the current date

for date, image in new.items():
    print( {date}, {image})


In [ ]:
#only run to make a copy of extracted unique images to new folder
import os
import shutil

# Base directory where new folders will be created
base_directory = '2D extracted frames'  # Replace with your desired path

# Iterate over the dictionary
for date, image_path in new.items():
    # Create a valid directory name for the date
    # Replace invalid characters for folder names (like '/') with an underscore or another valid character
    folder_name = date.replace(':', '_').replace('/', '_')
    directory_path = os.path.join(base_directory)

    # Create the directory if it doesn't exist
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Define the destination path for the image
    destination_path = os.path.join(directory_path, os.path.basename(image_path))

    # Copy the image to the new location
    shutil.copy(image_path, destination_path)

    print(f"Copied {image_path} to {destination_path}")


In [ ]:
pip install google-cloud-aiplatform

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project="crafty-ring-411412")


In [ ]:
pip install --upgrade google-cloud-aiplatform

In [ ]:
pip install protobuf==3.20

In [3]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

import vertexai
vertexai.init(project="crafty-ring-411412", location="us-central1")

In [4]:
multimodal_model = GenerativeModel("gemini-pro-vision")

In [5]:
#Sorting the frame names and Extracting frames from the folder.
import os
import re
from vertexai.preview.generative_models import Image

def extract_number(file_name):
    # Extract the numeric part from the filename
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else 0

# Directory containing your images
directory = "./2D extracted frames"

# List all image filenames in the directory
image_filenames = [filename for filename in os.listdir(directory) if filename.endswith(".jpg")]

# Sort the image filenames by their numeric part
sorted_filenames = sorted(image_filenames, key=lambda x: extract_number(x))
print(sorted_filenames)
# Load images in sorted order
images = [Image.load_from_file(os.path.join(directory, filename)) for filename in sorted_filenames]

print(len(images))


['frame_0.jpg', 'frame_133.jpg', 'frame_166.jpg', 'frame_205.jpg', 'frame_229.jpg', 'frame_254.jpg', 'frame_292.jpg', 'frame_321.jpg', 'frame_345.jpg', 'frame_367.jpg', 'frame_389.jpg', 'frame_413.jpg', 'frame_435.jpg', 'frame_464.jpg', 'frame_487.jpg', 'frame_494.jpg', 'frame_504.jpg', 'frame_515.jpg', 'frame_537.jpg', 'frame_543.jpg', 'frame_564.jpg', 'frame_586.jpg', 'frame_605.jpg', 'frame_630.jpg', 'frame_653.jpg', 'frame_678.jpg', 'frame_704.jpg', 'frame_729.jpg', 'frame_751.jpg', 'frame_770.jpg', 'frame_795.jpg', 'frame_821.jpg', 'frame_844.jpg', 'frame_875.jpg', 'frame_899.jpg', 'frame_916.jpg', 'frame_936.jpg', 'frame_955.jpg', 'frame_973.jpg', 'frame_1000.jpg', 'frame_1015.jpg', 'frame_1038.jpg', 'frame_1067.jpg', 'frame_1109.jpg', 'frame_1144.jpg', 'frame_1164.jpg', 'frame_1191.jpg', 'frame_1221.jpg', 'frame_1246.jpg', 'frame_1248.jpg', 'frame_1266.jpg', 'frame_1298.jpg', 'frame_1318.jpg', 'frame_1338.jpg', 'frame_1355.jpg', 'frame_1375.jpg', 'frame_1395.jpg', 'frame_1415.jp

In [19]:
#multiple images per request
New = images[:15]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.4,
    top_k=10,
    candidate_count=1,
    max_output_tokens=2048,
)
generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content([ *New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
print(response.candidates[0].text)

 | Date         | Open  | High | Low  | Close |
|-------------|-------|-------|-------|-------|
| 12/22/2022   | 3834.36 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3855.29 | 3893.46 | 3794.33 | 3852.97 |
| 12/28/2022   | 3836.08 | 3874.16 | 3802.42 | 3835.08 |
| 12/29/2022   | 3839.74 | 3905.93 | 3824.12 | 3890.09 |
| 12/30/2022   | 3910.82 | 3950.57 | 3897.29 | 3915.25 |
| 1/3/2023     | 3906.6  | 4013.23 | 3896.54 | 3999.09 |
| 1/4/2023     | 3928.28 | 4015.48 | 3926.59 | 3982.86 |
| 1/5/2023     | 3911.84 | 3972.96 | 3885.54 | 3927.61 |
| 1/6/2023     | 3978.14 | 4039.23 | 3957.64 | 4016.95 |
| 1/9/2023     | 4032.71 | 4061.57 | 3949.06 | 4060.43 |
| 1/10/2023    | 4055.72 | 4092.41 | 4015.55 | 4017.77 |
| 1/11/2023    | 4020.85 | 4148.95 | 4020.44 | 4119.21 |
| 1/12/2023    | 4158.68 | 4195.44 | 4123.36 | 4136.48 |


In [7]:
# #one image per repuest
# import time
# for image in New:
#     response = generative_multimodal_model.generate_content([*New,"parse date, open, high, low, close from this image and make a table", image])
#     print(response.candidates[0].text)
#     time.sleep(10)

In [12]:
#video preprocessing
import base64

def convert_video_to_base64(video_file_path):
    # Read the video file
    with open(video_file_path, 'rb') as video_file:
        video_data = video_file.read()

    # Encode the video data in base64
    base64_encoded_data = base64.b64encode(video_data)

    # Convert to a string for easier handling
    base64_encoded_string = base64_encoded_data.decode('utf-8')

    return base64_encoded_string

# Example usage
video_base64 = convert_video_to_base64('C:/Users/SUYASH/Downloads/downloaded_video.mp4') #replace video location here



In [18]:
prompt = """
parse all different reading of date, open, high, low, close, from this video.
"""
generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    }
video = Part.from_data(
    data=video_base64,
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)


print("\n-------Response--------")
for response in responses:
    print(response.text, end="")


-------Response--------
 The video shows the following readings:

Date	Open	High	Low	Close
2023-03-08	3846.65	4126.43	4039.56	4377.26
2023-03-09	4126.43	4439.56	4312.38	4772.94
2023-03-10	4439.56	4567.89	4439.56	4512.36
2023-03-13	4567.89	4634.50	4512.36	4600.12
2023-03-14	4634.50	4754.23	4600.12	4723.45